# Homework 2

### This is the solution of:
* Student1
* Student2
* Student3


The same general rules as for Homework 1 applies.

$\newcommand{\dx}{\,\mathrm{d}x}$

## Problem 1 (Finite Difference Method in 2D)

 **a)** Let $\Omega = (0,1)\times (0,1)$.
For the functions $u_k(x,y) = \sin(2 \pi k x) \sin(2 \pi k y)$ with frequency $k \in \mathbb{N}$,
compute analytically the right-hand side $f$ and boundary data $g$
such that $u_k$ satisfies the Poisson problem
\begin{align*}
- \Delta u_k  &= f \quad \text{in } \Omega,
\\
 u_k &= g \quad \text{on } \partial \Omega.
\end{align*}

**b)** Based on the code snippets below, implement a finite difference scheme to solve the problem given in a) for $k = 2$ numerically. For the computational grid, assume equally spaced subdivisions in $x$ and $y$ direction, starting with $N=8$ subintervals in each spatial direction. For $N = 8, 16, 32, 64$, compute and plot the finite difference solution $U$.

### Code Snippets

As in Lab 1, places marked with $\ldots$ need your attention and must be filled with proper code.

We start with importing the necessary scientific libraries and define name aliases for them.

In [ ]:
# Array and stuff 
import numpy as np
# Linear algebra solvers from scipy
import scipy.linalg as la
# Basic plotting routines from the matplotlib library 
import matplotlib.pyplot as plt
# We also need access to the colormaps for 3D plotting
from matplotlib import cm

Next, we define a surface plotting function.

In [ ]:
def plot2D(X, Y, Z, title=""):
    # Define a new figure with given size 
    fig = plt.figure(figsize=(11, 7), dpi=100)
    ax = fig.add_subplot(111, projection='3d') 
    surf = ax.plot_surface(X, Y, Z,             
                           rstride=1, cstride=1, # Sampling rates for the x and y input data
                           cmap=cm.viridis)      # Use the new fancy colormap viridis
    # Set initial view angle
    ax.view_init(30, 225)
    
    # Set labels and show figure
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_title(title)
    plt.show()

Now the real fun begins. We will implement our FDM in a function which takes the number of subintervals in each spatial direction $N$ and the frequency $k$ as arguments. We include a description of the function (a so-called "docstring") at the beginning of the function. To get a feeling for the various data structure we are going to use, it might be helpful to print out a few of them and to look them up in the numpy/scipy documentation while you implementing the scheme!

In addition to the inbuilt documentation in the Spyder IDE, you can also look up things easily by scanning through the method/class index at
https://docs.scipy.org/doc/numpy/genindex.html

In [ ]:
def fdm_poisson_2d_dense(N, k):
    '''A simple finite difference solver in 2d using a full matrix representation.'''

    # 1) Compute right hand side     
    
    # To define the grid we could use "linspace" as in Lab 1 to define subdivisions for the $x$ and $y$ axes.
    # But to make plotting easy and to vectorize the evaluation of the right-hand side $f$,  
    # we define x and y coordinates for the grid using a "sparse grid" representation using the function 'ogrid'.
   
    x,y = np.ogrid[0:1:(N+1)*1j, 0:1:(N+1)*1j]
    # Here, 1j is not interpreted as the imaginary unit, rather "(N+1)*1j" forces to include the end points
    # Print x and y to see how they look like!
    # print(x)
    # print(y)
    
    # Evaluate f on the grid. 
    F_grid = ...
    
    # You can print F_grid to verify that you got a 2 dimensional array
    # print(F_grid)
    
    # You can also plot F_grid now if you want :)
    # plot2D(x, y, F_grid, "f")
    
    # Now we define our rhs by flattening out F, making it a 1 dimensional array of length (N+1)*(N+1). 
    F = F_grid.ravel()  
    
    # 2) Create Matrix entries for unknowns associated with inner grid points. 
    
    # To translate the grid based double index into a proper numbering, we define a 
    # small mapping function, assuming a row-wise numbering. 
    # Drawing a picture of the grid and numbering the grid points in a row wise manner
    # helps to understand this mapping!
    def m(i,j):
        return ...
    
    # Total number of unknowns is M = (N+1)*(N+1)
    M = (N+1)**2
    
    # Allocate a (full!) MxM matrix filled with zeros
    A = ...
    
    # Meshsize h
    h = 1/N
    hh = h*h
    
    # Compute matrix A entries by iterating over the *inner* grid points first.
    for i in range(1,N):      # i is the row number for the grid point
        for j in range(1,N):  # j is the column number for the grid point
            # Compute the index of the unknown at grid point (i,j). 
            # This is also the index of the row in matrix A we want to fill. 
            ri = m(i,j)       
            A[ri,...] = ... # U_ij
            A[ri,...] = ...   # U_{i-1,j}
            A[...,...] = ...    # U_{i+1,j}
            A[...,...] = ...    # U_{i,j-1}
            A[...,...] = ...    # U_{i,j+1}
    
    # 3) Incorporate boundary conditions
    # Boundary condition 
    for i in [0, N]:
        for j in range(0,N+1):
            # Define row index related to unknown U_m(i,j)
            ri = ...
            A[...,...] = 1 
            F[...] = 0     
    
    # Boundary condition 
    for j in [0, N]:
        for i in range(0,N+1):
            # Define row index related to unknown U_m(i,j)
            ri = ...
            A[...,...] = 1 
            F[...] = 0     
    
    
    # 4) Solve linear systems
    # Solve linear algebra system 
    U = ...
    
    # Reshape the flat solution vector U to make it a grid function
    U_grid = U.reshape((N+1,N+1))
    
    # Return solution and x and y grid points for easy plotting
    return (x,y,U_grid)

**c**) Compute the experimental order of convergence (EOC)
for $N = 16, 32, 64$ using $\max_{i} |U-u|$ as error measure. Summarize your results in a table. 
What convergence rate do you get? If you don't get an EOC very close to $2$, find the bugs in your code :)

**d**) Test how large you can choose the resolution $N$ until either the problem takes too long (say 5 minutes) to compute or uses too much memory. Explain, what happens. Why does the problem
scale so badly with respect to $N$?

**e**) Based on your implementation, we now implement a finite difference using *sparse* matrices. Knowing the structure and entries of the matrix a priori, the most efficient 
realization would be based on (block) tridiagonal sparse matrices. 
But anticipating the forthcoming task of implementing schemes based on the finite element method, we will take a middle ground and simply switch to a flexible sparse matrix format which allows
for minimal adjustments of your previous solver implementation.
To this end, you shall incorporate the following code snippets into your code.

### Code Snippets

Get access to sparse matrices and sparse solvers.

In [ ]:
import scipy.sparse as sp
from scipy.sparse.linalg import spsolve

Use a sparse matrix format for $A$, see https://docs.scipy.org/doc/scipy/reference/sparse.html
for the many formats which are available. Here we use "dictionary of keys" based representation which
is an empty matrix to begin with and which can easily filled with non-zero values at the appropriate
places.

In [ ]:
A = sp.dok_matrix((M, M))

After creating the matrix we have to convert it to a different format, the so-called
"Compressed Sparse Row matrix" representation, which is much more efficient when solving the system $A U = F$ with a sparse solver.

In [ ]:
# Now convert A to format which is more efficient for solving
A_csr = A.tocsr() 
U = spsolve(A_csr, F)

**f**) Measure and compare the overall solution time for your two implementations 'fdm_poisson_2d_dense' and 'fdm_poisson_2d_sparse' by using the cell magic command %%timeit.
Here is a simple example of its usage. Simply execute the next cell.

In [ ]:
%%timeit # Measure execution time for entire cell, has to be at the top of the cell.

def compute_sum(N):
    sum = 0
    for i in range(0,N):
        sum += i
    return sum

N = 10000000
compute_sum(N)

In [ ]:
# If you want to measure execution time for a single line write %timeit instead
%timeit compute_sum(N)

## Problem 2 (Quadrature Rules)

**a)** Write computer functions which compute the integral $\int_a^b f(x) \dx$ 
for a given $f$ and interval $[a,b]$ using the mid-point, trapezoidal and Simpson's rule.

**b)** For the monomial functions $p_i(x) = x^i$ and $i = 1,2,3$,
compute the integral $\int_0^1 p_i(x) \dx$ numerically using all three quadrature schemes.
Compute the exact integrals analytically and compute the quadrature error
$$
\text{err}(p_i, Q) = \left| \int_I p_i \dx - Q(p_i, I) \right|
$$

for each combination of quadrature rule and monomial. Summarize your result in
a table (quadrature rules as column header, monomials as row headers. You may use pandas library, see below.) Why does some quadrature error vanish? 

In [1]:
import pandas as pd

data = {'midpoint': [0, 0, 0],
        'Trapezoid': [0, 0, 0],
        'Simpson': [0, 0, 0]
        }

df = pd.DataFrame(data, index=['monomial_order_1','monomial_order_2','monomial_order_3'])

print (df)

                  midpoint  Trapezoid  Simpson
monomial_order_1         0          0        0
monomial_order_2         0          0        0
monomial_order_3         0          0        0


In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def mid_point(f, a, b, N):
    h = (b - a)/N
    g = 0
    for i in range(N):
        xm = h*(i + 0.5) + a
        g += f(xm)*h
    
    return g

def simpsons(f, a, b, N):
    h = (b - a)/N
    g = 0
    x = np.linspace(a, b, N+1)
    for i in range(1, N+1):
        xm = 0.5*(x[i-1] + x[i])
        g += h/6*(f(x[i-1]) + f(x[i]) + 4*f(xm))

    return g

f = lambda x: x**2
g = mid_point(f, 0, 1, 100)
gs = simpsons(f, 0, 1, 100)

print(g)
print(gs)
    

0.995
0.33332500000000004
0.3333333333333333


**c)** Divide $[0,1]$ into 2, 4 and then 8 equally spaced subintervals.
For those combinations of quadrature rules/monomials where the quadrature error does not vanish, compute a better approximation of the integral $\int_0^1 p_i(x) \dx$
by applying the corresponding quadrature rule separately on each of the 2, 4, 8 subintervals.
What experimental order of convergence do you observe for the quadrature rule/monomial pairs? 

## Problem 3 (Piecewise Linear Interpolation)

Let $0 = x_0 < x_1 < x_2 < \ldots < x_{N} = 1$ be a partition of the interval
$0\leq x\leq 1$ into $N$ subintervals of equal length $h$.  Moreover,
let $\{ \varphi_j\}_{j=0}^{N}$ be the set of hat basis functions of $V_h$
associated with the $N+1$ nodes $x_j$, $j = 0,1\ldots, N$, such that
\begin{align}
  \varphi_i(x_j) =
  \left \{ 
  \begin{array}{l}
    1, \quad \mbox{if } i = j, \\
    0, \quad \mbox{if } i \neq j.
  \end{array}
  \right .
\end{align}
The explicit expression for a hat function $\varphi_i(x)$ is given by 
\begin{align} 
  \varphi_i(x) =
  \left\{
  \begin{array}{ll}
    (x-x_{i-1})/h, &\mbox{if } x_{i-1} \leq x \leq x_i,\\
    (x_{i+1}-x)/h, & \mbox{if } x_i \leq x \leq x_{i+1},\\
    0, & \mbox{otherwise.} 
  \end{array}
  \right. 
\end{align}

**a)** Write a Python function that computes and returns the hat functions $\varphi_i$, $i=0,1,\dots,N$, where ${\texttt{xn}}$ is a vector containing the $N+1$ nodal points,
and $\texttt{x}$ is an array of points you want to evaluate hat function
$\varphi_i$ at. Then, plot $\varphi_2$ and $\varphi_N$ in partitions with $N=4,7,10$. (Use a finer sampling of $[0,1]$ than given by the nodal points for the plotting!)

### Code Snippet

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def hatfun(xn, i, x):
    y = np.zeros(x.size)
    N = xn.size-1
    
    # Make indexing work for zero dim array
    if x.ndim == 0:
        x = np.array([x])
    
    for j in range(0, y.size):
        # Left boundary
        if i == 0:
            if  xn[0] <= x[j] and x[j] <= xn[1]:
                y[j] = (xn[1] - x[j])/(xn[1] - xn[0])
        # Right boundar
        elif i == N:
            if  ...:
                y[j] = ...
        # Interior point
        elif xn[i-1] <= x[j] and x[j] <= xn[i+1]:
            if  ... :
                y[j] = ...
            else:
                y[j] = ...
    return y


**b**) Write a Python script ${\texttt { interp1d(f, xn, x)}}$, that computes the linear interpolant $\pi f_k\in V_h$, $k=1,2,3$ of 

* $f_1(x)=x\sin (3\pi x)$
* $f_2(x)=2-10x$
* $f_3(x)=x(1-x)$

by using your function ${\texttt {hatfun}}$. 

Hint: Recall that the interpolant is defined by
$$
\pi f(x) = \sum_{i=0}^{N} f(x_i) \varphi_i(x)
$$
Compute the error in the numerical solution using the $L^2$-norm
$
    \left(\| v \|_{L^2(I)}= \left( \int_I v^2 \right)^{1/2}\right)
$
and present the results in Log-Log plots (error versus $h$) using partitions with $N=2^2,2^3,...,2^6$. Do the errors behave as  
$$
\| f - \pi f \|_{L^2(I)}^2 \leqslant C \sum_{i=0}^{N} h_i^4 \| f'' \|_{L^2(I_i)}^2
$$
for all cases? If not, explain why. 
*Hint*: Use the code for the Simpson's formula to compute the $L^2$-norm per interval in the partition.